Script for analyzing MCell molecule location data for calcium diffusion

Last updated: 09/08/21

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import seaborn as sns
from scipy import stats
from math import pi

In [11]:
def ca_snare_finder(dir,iter_num, plot=True):
    '''
    Calculates concentration of calcium in a box around the SNARE complexes
    at a given time for MCell simulation.

    :param dir: MCell ASCII data location
    :param iter_num: which iteration/time point to plot
    :param plot: if output should be plotted
    :return: ca radial locations
    '''   
    
    # SNARE box dimensions (using refrac box dimensions) (TODO: find out if half len or full)
    '''
    x_len = 0.04
    y_len = 0.035
    z_len = 0.005
    '''
    
    x_len = 0.08
    y_len = 0.07
    z_len = 0.01
    
    
    #rxn_rad = 0.02     # reaction radius size (micron)

    # SNARE BOX LOCATION
    # for a giant box snare complex 
    #z = -0.25
    z = -.24550 # using refrac box location
    snare_locs = [(0, 0, z)] # for only one giant SNARE (x, y, z)

    
    '''
    snare_locs = [(0, 0),
                  (0.035, 0),
                  (-0.035, 0),
                  (0.0175, 0.0303),
                  (-0.0175, 0.0303),
                  (0.0175, -0.0303),
                  (-0.0175, -0.0303)]      # SNARE (x,y) locations
    '''

    ca_hist = []
    for seed in sorted(os.listdir(mcell_viz_dir)):

        # data file location
        iter_file = os.path.join(mcell_viz_dir, seed, 'Scene.ascii.{}.dat'.format(iter_num))

        # create dataframe
        loc_data = pd.read_csv(iter_file, delim_whitespace=True, header=None,
                           names=['type', 'id', 'x', 'y', 'z', 'norm_x', 'norm_y', 'norm_z'])

        # select only calcium
        loc_data = loc_data[loc_data['type'] == 'ca']

        # radius from x, y, and z coordinates
        #loc_data['r'] = np.sqrt(loc_data['x'] ** 2 + loc_data['y'] ** 2 + loc_data['z'] ** 2)

        snare_ca_tot = 0        # number of calcium in SNARE rxn zone

        # ONLY 1 SNARE TO START
        for x,y,z in snare_locs:
            #print("SNARE LOCATION (x,y,z):\t", x,y,z)

            '''
            # ranges
            print('x range',(x - x_len), (x + x_len))
            print('y range',(y - y_len), (y + y_len))
            print('z range',(z - z_len), (z + z_len))
            '''

            # Ugly method (TODO: make nicer)
            # get calcium location (x,y,z)
            for ca_x, ca_y, ca_z in zip(loc_data['x'], loc_data['y'], loc_data['z']):
                # check z direction
                if (ca_z >= z - z_len) & (ca_z <= z + z_len):
                    #print('z match:', ca_x, ca_y, ca_z)
                    # check x direction
                    if (ca_x >= x - x_len) & (ca_x <= x + x_len):
                        #print('x and z match:', ca_x, ca_y, ca_z)
                        # check y direction
                        if (ca_y >= y - y_len) & (ca_y <= y + y_len):
                            #print("CA LOCATION:\t", ca_x, ca_y, ca_z)
                            snare_ca_tot += 1
            print(snare_ca_tot)

            '''
            # Slick method
            snare_ca_tot += len(loc_data[(loc_data['x'] >= x - rxn_rad) & (loc_data['x'] <= x + rxn_rad) 
                    & (loc_data['y'] >= y - rxn_rad) & (loc_data['y'] <= y + rxn_rad) 
                    & (loc_data['z'] <= z + rxn_rad)])
            '''

        #print()
        if snare_ca_tot != 0:
            print("TIME POINT:\t", iter_num)
            print("TOTAL NUMBER:\t\t", snare_ca_tot)
            conc_ca = snare_ca_tot/ (x_len * y_len * z_len)
            print("TOTAL CONC (# ca/um^3):\t\t", conc_ca)
            conc_ca_M = conc_ca / (1e-15 * 6.022e23)
            print("TOTAL CONC (M):\t", conc_ca_M)
            #print("TOTAL CONC:\t\t", (2*snare_ca_tot*pi*(rxn_rad**3))/3) # TODO: CHANGE TO REFLECT VOLUME OF INTEREST
            print()
            print()

    '''
    Plotting    
        for value in loc_data['r'].values:
            ca_hist.append(value)

    if plot:
        #plt.hist(ca_hist, bins=250, color='C0')
        sns.distplot(ca_hist, bins=100)
        plt.show()

    return ca_hist
    '''

In [3]:
# plot for multiple time points
mcell_viz_dir = "/Users/margotwagner/ucsd/research/DiffusionModel/" \
                "rect_zeroflux_nocalbpmca_files/mcell/output_data/viz_data_ascii"

In [4]:
# get time points
time_pts = [scene.split('.')[2] for scene in os.listdir(os.path.join(mcell_viz_dir, 'seed_00001'))]
time_pts.sort()

In [5]:
#sample points
nums = [time_pts[i] for i in range(len(time_pts)) if i%1==0]
print(nums)

['0000', '0005', '0010', '0015', '0020', '0025', '0030', '0035', '0040', '0045', '0050', '0055', '0060', '0065', '0070', '0075', '0080', '0085', '0090', '0095', '0100', '0105', '0110', '0115', '0120', '0125', '0130', '0135', '0140', '0145', '0150', '0155', '0160', '0165', '0170', '0175', '0180', '0185', '0190', '0195', '0200', '0205', '0210', '0215', '0220', '0225', '0230', '0235', '0240', '0245', '0250', '0255', '0260', '0265', '0270', '0275', '0280', '0285', '0290', '0295', '0300', '0305', '0310', '0315', '0320', '0325', '0330', '0335', '0340', '0345', '0350', '0355', '0360', '0365', '0370', '0375', '0380', '0385', '0390', '0395', '0400', '0405', '0410', '0415', '0420', '0425', '0430', '0435', '0440', '0445', '0450', '0455', '0460', '0465', '0470', '0475', '0480', '0485', '0490', '0495', '0500', '0505', '0510', '0515', '0520', '0525', '0530', '0535', '0540', '0545', '0550', '0555', '0560', '0565', '0570', '0575', '0580', '0585', '0590', '0595', '0600', '0605', '0610', '0615', '0620',

In [12]:
# For one timepoint
ca_snare_finder(mcell_viz_dir, '0310', plot=True)

8
TIME POINT:	 0310
TOTAL NUMBER:		 8
TOTAL CONC (# ca/um^3):		 142857.14285714284
TOTAL CONC (M):	 0.00023722541158608903




In [30]:
# All time points
for n in nums:
    time_hist(mcell_viz_dir, n, plot=True)


0

0

0

0

0

x and z match: -0.000330552543 -0.0400021558 -0.245829587
0

0

x and z match: 0.0049102244 0.16423319 -0.24150429999999998
x and z match: 0.0117770603 0.0658119246 -0.24288331300000002
x and z match: 0.0335317184 -0.186314531 -0.24576340300000002
x and z match: 0.0372640811 -0.10409176 -0.248818873
0

x and z match: -0.00164590664 0.18716275 -0.247672347
0

x and z match: -0.0021837659 0.179917319 -0.244287444
x and z match: 0.0047520101200000005 0.216447217 -0.241190359
x and z match: -0.0397005906 0.0681365477 -0.247474501
x and z match: 0.015701346 0.160719538 -0.248650548
x and z match: 0.0123747199 -0.12131196 -0.242285998
0

x and z match: -0.024140011200000002 0.0528025584 -0.24652441100000003
x and z match: 0.0192063751 -0.164423939 -0.24632529399999997
0

x and z match: 0.0252615344 -0.181736297 -0.24118545600000002
0

x and z match: 0.0111865944 0.11494602400000001 -0.247532802
x and z match: -0.0136681095 0.081759134 -0.249456928
x and z match: 0.0109744655 

In [21]:
for i in ['0380', '0385', '0390', '0395', '0400', '0405', '0410', '0415', '0420', '0425', '0430', '0435', '0440', '0445', '0450', '0455', '0460', '0465', '0470']:
    print(i)
    time_hist(mcell_viz_dir, i, plot=True)

0380

x range -0.04 0.04
y range -0.035 0.035
z range -0.2505 -0.2405
1.60267035 0.163709677 -0.173823256
0.945908163 0.20066193100000002 -0.23221692100000002
1.04846601 0.244170579 -0.233182203
1.07406508 0.160102412 -0.24669782
0.8235982070000001 0.13187141 -0.247215944
0.993716508 0.0946839926 -0.249934143
0.50086423 -0.016069983 -0.249938104
-0.091014077 -0.030121295699999998 -0.24996075
-0.24996075
0385

x range -0.04 0.04
y range -0.035 0.035
z range -0.2505 -0.2405
1.6221811000000002 0.20537377899999998 -0.153548425
1.49074174 0.138893996 -0.245045486
0.8819239529999999 0.204080779 -0.245883723
1.16152067 0.08925267449999999 -0.247405121
0.222875105 0.179035716 -0.24779550399999997
0.43504244799999997 0.173443417 -0.248419253
0.104151825 0.136696226 -0.24863211
0.446097841 0.20592262600000003 -0.249398838
0.09168956550000001 0.23849151600000001 -0.249940855
-0.35345611200000004 0.0601028892 -0.249964107
0.232235429 -0.121281004 -0.249982526
-0.249982526
0390

x range -0.04 0.04
